In [1]:
using LinearAlgebra
using JuMP
using MathOptInterface
using Dualization
using Convex
using MosekTools
using Plots
using Statistics
using ControlSystems
using Dates
using DelimitedFiles

┌ Info: Precompiling JuMP [4076af6c-e467-56ae-b986-b466b2749572]
└ @ Base loading.jl:1273
┌ Info: Precompiling Dualization [191a621a-6537-11e9-281d-650236a99e60]
└ @ Base loading.jl:1273
┌ Info: Precompiling ControlSystems [a6e380b2-a6ca-5380-bf3e-84a91bcd477e]
└ @ Base loading.jl:1273


---------------------------

In [2]:
# The non-block dynamics of an LTI system
struct Lti
    n::Int64 # State dimension
    m::Int64 # Control dimension
    A::Matrix{Float64} # n x n matrix
    B::Matrix{Float64} # n x m matrix
    sigma::Float64
    Lti(A, B, sigma) = (
        A = Matrix{Float64}(A);
        B = Matrix{Float64}(B);
        sigma = Float64(sigma);
        (ah, aw) = size(A); @assert ah == aw;
        (bh, bw) = size(B); @assert bh == aw;
        new(ah, bw, A, B, sigma)
    )
end

# The Ocal matrix
function Ocal(L::Int64, X)
    (xh, xw) = size(X)
    @assert xh == xw
    M = zeros(0, xw)
    for t = 0:L-1; M = [M; X^t] end
    return Matrix{Float64}(M)
end

# The Tcal matrix
function Tcal(lti::Lti, L::Int64, X)
    n = lti.n
    A = lti.A
    (xh, xw) = size(X)
    @assert n == xh
    T = zeros(n * L, 0)
    Z = Matrix{Float64}(kron(Bidiagonal(zeros(L), ones(L - 1), :L), I(n)))
    for t = 1:L; T = [T (Z^t * Ocal(L, A) * X)] end
    return Matrix{Float64}(T)
end

# Block dynamics parameterized by a data of type Lti
struct BlockLti
    lti::Lti
    L::Int64
    Acal::Matrix{Float64} # nL x nL block matrix
    Bcal::Matrix{Float64} # nL x mL block matrix
    Zcal::Matrix{Float64} # nL x nL block matrix
    BlockLti(lti::Lti, L::Int64) = (
        In = Matrix{Float64}(I(lti.n));
        Acal = kron(I(L), lti.A);
        Bcal = kron(I(L), lti.B);
        Zcal = Matrix{Float64}(kron(Bidiagonal(zeros(L), ones(L - 1), :L), In));
        new(lti, L, Acal, Bcal, Zcal)
    )
end
;

In [3]:
# Explicitly store information about a Hankel matrix in a struct
struct Hankel
    d::Int64 # Dimension of underlying signal
    L::Int64
    T::Int64
    H::Matrix{Float64} # dL x (T - L + 1) dimensional block matrix
    Hankel(d::Int64, H) = (
        H = Matrix{Float64}(H);
        (hh, hw) = size(H);
        @assert mod(hh, d) == 0;
        L = Int64(hh / d);
        T = hw + L - 1;
        new(d, L, T, H)
    )
end

# Generate a trajectory of states starting from x0,
# given a control sequence u and noise sequence w,
# with respect to the dynamics given by lti::Lti
function xtraj(lti::Lti, x0::Array{Float64}, u::Array{Float64}, w::Array{Float64})
    n = lti.n; m = lti.m; A = lti.A; B = lti.B
    
    @assert length(x0) == n
    @assert length(u) / m == length(w) / n
    @assert mod(length(u), m) == 0
    @assert mod(length(w), n) == 0
    
    T = Int64(length(u) / m)
    
    x = zeros(n*T)
    x[1:n] = x0
    for t = 0:(T-2)
        (tlo, thi) = ((n * t) + 1, n * (t + 1))
        x[(tlo + n):(thi + n)] = A * x[tlo:thi] + B * u[tlo:thi] + w[tlo:thi]
    end
    
    return x
end

# We use z to denote system states _without_ process noise,
# so this merely passes w = zeros(n * T) to the xtraj function
function ztraj(lti::Lti, z0::Array{Float64}, u::Array{Float64})
    @assert mod(length(u), lti.m) == 0
    T = Int64(length(u) / lti.m)
    return xtraj(lti, z0, u, zeros(lti.n * T))
end

# Generate the Hankel data struct, assuming z is a stacked vector with block dimension d
function Hankelize(z::Array{Float64}, d::Int64, L::Int64)::Hankel
    N = Int64(round(length(z) / d))
    @assert length(z) == N * d
    z = reshape(z, (d, N))
    H = Matrix{Float64}(zeros(d * L, N - L + 1))
    for i = 1:(N - L + 1); H[:, i] = (z[:, i:(i + L - 1)])[:] end
    return Hankel(d, H)
end

# The Wcal matrices for relating the noisy states Hx to the noiseless Hz
function W(lti::Lti, t::Int64, w::Array{Float64})
    n = lti.n
    A = lti.A
    
    @assert mod(length(w), n) == 0
    
    acc = zeros(n)
    for k = 0:t-1
        (wlo, whi) = ((n * k) + 1, n * (k + 1))
        acc = acc + A^(t - 1 - k) * w[wlo:whi]
    end
    return acc
end
;

In [4]:
# System dynamics of the slightly unstable graph Laplacian
lti = Lti(
    # A
    [ 1.01 0.01 0.00;
      0.01 1.01 0.01;
      0.00 0.01 1.01 ],
    
    # B
    Matrix{Float64}(I(3)),
    
    # sigma
    sqrt(0.1)
)

# Hankel matrix parameters
L = 10
T = 45
blk = BlockLti(lti, L)
;

In [5]:
# Sanity check things about the (block) system dynamics;
# Will cause an assertion failure if something is wrong
function sanitycheck()
    N = 2000
    x0 = [10.0; 10.0; 10.0]
    
    u = zeros(lti.m * T)
    w = zeros(lti.n * T)
    x = zeros(lti.n * T)
    z = zeros(lti.n * T)
    
    for k = 1:N
        ku = randn(lti.m * T)
        kw = randn(lti.n * T) * lti.sigma
        kx = xtraj(lti, x0, ku, kw)
        kz = ztraj(lti, x0, ku)
        
        u = u + ku
        w = w + kw
        x = x + kx
        z = z + kz
    end
    
    u = u / N
    x = x / N
    z = z / N
    w = w / N
    
    # Toggle to use a single noise trajectory instead
#     u = randn(lti.m * T)
#     w = randn(lti.n * T) * (lti.sigma / sqrt(N))
#     x = xtraj(lti, x0, u, w)
#     z = ztraj(lti, x0, u)
    
    
    Hankx = Hankelize(x, lti.n, blk.L)
    Hankz = Hankelize(z, lti.n, blk.L)
    Hanku = Hankelize(u, lti.m, blk.L)
    Hankw = Hankelize(w, lti.n, blk.L)
    
    
    epsT = opnorm(Hankw.H)
    println("epsT = " * string(epsT))

    Wcal = zeros(lti.n, 0)
    for t = 0:T-L; Wcal = [Wcal W(lti, t, w)]end

    Eyen = Matrix{Float64}(I(lti.n))

    @assert norm(Hankz.H + Tcal(lti, L, Eyen) * Hankw.H + Ocal(L, lti.A) * Wcal - Hankx.H) < 0.00001
end

sanitycheck()

epsT = 0.06512504792955785


--------------------

In [6]:
# The LQR cost data structure
struct Lqr
    n::Int64
    m::Int64
    Q::Matrix{Float64}
    R::Matrix{Float64}
    sigma::Float64
    Lqr(Q, R, sigma) = (
        Q = Matrix{Float64}(Q);
        R = Matrix{Float64}(R);
        sigma = Float64(sigma);
        (qh, qw) = size(Q); @assert qh == qw;
        (rh, rw) = size(R); @assert rh == rw;
        new(qh, rh, Q, R, sigma)
    )
end

# The LQR, except in block form, so the Qcal, Rcal etc matrices are also stored
# This distinction is made to reflect the Lti vs BlockLti structs earlier
struct BlockLqr
    lqr::Lqr
    L::Int64
    Qcal::Matrix{Float64}
    Rcal::Matrix{Float64}
    sqrtQcal::Matrix{Float64}
    sqrtRcal::Matrix{Float64}
    BlockLqr(lqr::Lqr, L::Int64, QL, RL) = (
        @assert size(lqr.Q) == size(QL);
        (qh, qw) = size(QL);
        Qcal = kron(I(L), lqr.Q);
        Qcal[(end - qh + 1):end, (end - qw + 1):end] = QL;
        
        @assert size(lqr.R) == size(RL);
        (rh, rw) = size(RL);
        Rcal = kron(I(L), lqr.R);
        Rcal[(end - rh + 1):end, (end - rw + 1):end] = RL;
        
        sqrtQcal = sqrt(Qcal);
        sqrtRcal = sqrt(Rcal);
        new(lqr, L, Qcal, Rcal, sqrtQcal, sqrtRcal)
    )    
end
;

In [7]:
# Expression of the Block LQR cost, before the Frobenius norm is taken
function LqrExpr(blqr::BlockLqr, PhiX, PhiU)
    E = blqr.lqr.sigma * [blqr.sqrtQcal * PhiX; blqr.sqrtRcal * PhiU]
    return E
end

# Generate an Ocal(Z')' matrix as used in the paper
function OcalZ(blk::BlockLti)
    return Matrix{Float64}(Ocal(L, blk.Zcal')')
end

# Given the system (blk::BlockLti), costs (blqr::BlockLqr),
# and data matrices (Hx::Hankel, Hu::Hankel), synthesize a G.
# Additional parameters include whether we are doing robust synthesis
function findG(blk::BlockLti, blqr::BlockLqr, Hx::Hankel, Hu::Hankel; robust=false, epsilon=-1, gamma=0.0)
    # Problem dimensions
    n = blk.lti.n; m = blk.lti.m;
    L = Hx.L; T = Hx.T
    
    # Sanity check dimensions and whether robustness is enabled
    @assert (m, n) == (Hu.d, Hx.d)
    @assert (Hx.L, Hx.T) == (Hu.L, Hu.T)
    # robust == true implies (gamma > 0) and (epsilon > 0)
    @assert (!robust) || (gamma > 0 && epsilon > 0)
    
    # Swap these two lines to dualize the problem, or not
    model = Model(dual_optimizer(optimizer_with_attributes(
    # model = Model((optimizer_with_attributes(
            Mosek.Optimizer,
            # "QUIET" => false,
            "QUIET" => true,
            "INTPNT_CO_TOL_DFEAS" => 1e-9
        )))
    
    # Declare the G matrix
    (gh, gw) = (T - L + 1, n)
    @variable(model, G[1:(gh * L), 1:(gw * L)])
    
    # Declare a cost variable
    @variable(model, cost)
    
    # The PhiX and PhiU matrices can be defined in terms of G
    PhiX = OcalZ(blk) * kron(I(L), Hx.H) * G
    PhiU = OcalZ(blk) * kron(I(L), Hu.H) * G
    
    # Encode the objective cost this way because of JuMP's API
    @constraint(model, [cost; vec(LqrExpr(blqr, PhiX, PhiU))] in SecondOrderCone())
    
    # Block diagonals are in the Gamma set
    for k = 0:L-1
        # Diagonal is identity in the PhiX matrix
        (plo, phi) = (k * n + 1, k * n + n)
        @constraint(model, PhiX[plo:phi, plo:phi] .== Matrix{Float64}(I(n)))
        
        # If robustness is enabled, additionally enable opnorm constraints
        if robust
            (grlo, grhi) = (k * gh + 1, k * gh + gh)
            (gclo, gchi) = (k * gw + 1, k * gw + gw)
            Gk = G[grlo:grhi, gclo:gchi]
            gnorm = gamma / (epsilon * sqrt(L))
            # println("gnorm = " * string(gnorm))
            @constraint(model, [gnorm; vec(Gk)] in MOI.NormSpectralCone(gh, gw))
        end
    end
    
    # Block lower triangular structure
    for i = 0:(L - 1)
        for j = (i + 1):(L - 1)
            (xrlo, xrhi) = (i * n + 1, i * n + n)
            (xclo, xchi) = (j * n + 1, j * n + n)
            @constraint(model, PhiX[xrlo:xrhi, xclo:xchi] .== zeros(n, n))
            
            (urlo, urhi) = (i * m + 1, i * m + m)
            (uclo, uchi) = (j * n + 1, j * n + n)
            @constraint(model, PhiU[urlo:urhi, uclo:uchi] .== zeros(m, n))
            
            (grlo, grhi) = (i * gh + 1, i * gh + gh)
            (gclo, gchi) = (j * gw + 1, j * gw + gw)
            @constraint(model, G[grlo:grhi, gclo:gchi] .== Matrix{Float64}(zeros(gh, gw)))
        end
    end
    
    # The strictly lower block triangles of G are also zero
    for i = 0:(L - 1)
        for j = 0:(i - 1)
            (grlo, grhi) = (i * gh + 1, i * gh + gh)
            (gclo, gchi) = (j * gw + 1, j * gw + gw)
            @constraint(model, G[grlo:grhi, gclo:gchi] .== Matrix{Float64}(zeros(gh, gw)))
        end
    end

    # For the robust problem, the  1/(1-gamma) part is done in the outer quasi-convex loop
    @objective(model, Min, cost)
        
    optimize!(model)
    return (model, value.(G))
end
;

In [8]:
## Non-robust synthesis
function findGstar(blk::BlockLti, blqr::BlockLqr, Hz::Hankel, Hu::Hankel)
    return findG(blk, blqr, Hz, Hu, robust=false)
end

# Identical to findGstar, except the parameter is named differently for semantics
function findGnom(blk::BlockLti, blqr::BlockLqr, Hx::Hankel, Hu::Hankel)
    return findG(blk, blqr, Hx, Hu, robust=false)
end

# Find a robust G wrt some epsilon > 0
function findGrob(blk::BlockLti, blqr::BlockLqr, Hx::Hankel, Hu::Hankel, epsilon)
    n = blk.lti.n; m = blk.lti.m
    L = Hx.L; T = Hx.T
    @assert (m, n) == (Hu.d, Hx.d)
    @assert (Hx.L, Hx.T) == (Hu.L, Hu.T)
    @assert epsilon > 0
    
    # Search range of gamma
    (left, right) = (1e-5, 1 - 1e-5)
    mid = (left + right) / 2
    
    # model and Grob are what we eventually return
    model = Model()
    Grob = zeros(n * L, T - L + 1)
    
    # Counters
    feasible = false
    iters = 0
    
    while (right - left > 1e-2)
        mid = (left + right) / 2
        iters = iters + 1
        (itermodel, iterG) = findG(blk, blqr, Hx, Hu, robust=true, epsilon=epsilon, gamma=mid)
        
        termstat = termination_status(itermodel)
        println("term stat: " * string(termstat) * " at bisection mid = " * string(mid))
        if termstat == MOI.OPTIMAL
            model = itermodel
            Grob = iterG
            right = mid
            feasible = true
        else
            left = mid
        end
    end
    
    println("Total iterations: " * string(iters))
    
    if !feasible
       println("Warning: problem was never feasible at epsilon = " * string(epsilon)) 
    end
    
    return (model, Grob, mid)
end

;

In [11]:
# Define the LQR costs
lqr = Lqr(1e-3 * Matrix{Float64}(I(lti.n)), Matrix{Float64}(I(lti.m)), lti.sigma)

# While we're at it, we can also examine the DARE solution
P = dare(lti.A, lti.B, lqr.Q, lqr.R)

# blqr = BlockLqr(lqr, blk.L, 4e2 * lqr.Q, lqr.R)
blqr = BlockLqr(lqr, blk.L, P, lqr.R)

;

In [12]:
# The LQR cost of a K matrix against the true system dynamics
function trueSystemLqrCost(blk::BlockLti, blqr::BlockLqr, K::Matrix{Float64})
    (kh, kw) = size(K)
    @assert kh == blk.lti.m * L
    @assert kw == blk.lti.n * L
    
    PhiXtrue = inv(I - blk.Zcal * (blk.Acal + blk.Bcal * K))
    J = blqr.lqr.sigma * norm(vec([blqr.sqrtQcal * PhiXtrue; blqr.sqrtRcal * K * PhiXtrue]))
    return J
end
;

In [20]:
function sanitycheck2()
    
    N = 10
    # x0 = [10.0; 10.0; 10.0]
    x0 = [0.0; 0.0; 0.0]
    u = zeros(lti.m * T)
    w = zeros(lti.n * T)
    x = zeros(lti.n * T)
    z = zeros(lti.n * T)
    
    for k = 1:N
        ku = randn(lti.m * T) * sqrt(lti.m * T * 1000)
        kw = randn(lti.n * T) * lti.sigma
        kx = xtraj(lti, x0, ku, kw)
        kz = ztraj(lti, x0, ku)
        
        u = u + ku
        w = w + kw
        x = x + kx
        z = z + kz
    end
    
    u = u / N
    w = w / N
    z = z / N
    x = x / N
    
    # Alternatively, generate data from a distribution
#     u = randn(lti.m * T)
#     w = randn(lti.n * T) * (lti.sigma / sqrt(N))
#     x = xtraj(lti, x0, u, w)
#     z = ztraj(lti, x0, u)
    
    println((norm(z), norm(x), norm(u), norm(w)))

    Hankx = Hankelize(x, lti.n, blk.L)
    Hankz = Hankelize(z, lti.n, blk.L)
    Hanku = Hankelize(u, lti.m, blk.L)
    Hankw = Hankelize(w, lti.n, blk.L)
    
    epsT = opnorm(Hankw.H)
    
    println("eps: " * string(epsT))
    println("opnorms (Hx, Hz, Hu, Hw) = " *
            string((opnorm(Hankx.H), opnorm(Hankz.H), opnorm(Hanku.H), opnorm(Hankw.H))))
    
    (model, Gstar) = findGstar(blk, blqr, Hankz, Hanku)
    PhiZstar = OcalZ(blk) * kron(I(L), Hankz.H) * Gstar
    PhiUstar = OcalZ(blk) * kron(I(L), Hanku.H) * Gstar
    Kstar = PhiUstar * inv(PhiZstar)

    (_, Gnom) = findGnom(blk, blqr, Hankx, Hanku)
    PhiXnom = OcalZ(blk) * kron(I(L), Hankx.H) * Gnom
    PhiUnom = OcalZ(blk) * kron(I(L), Hanku.H) * Gnom
    Knom = PhiUnom * inv(PhiXnom)

    @time (modelB, Grob, gammarob) = findGrob(blk, blqr, Hankx, Hanku, epsT)
#     @time (modelB, Grob) = findG(blk, blqr, Hankx, Hanku, robust=true, gamma=0.5, epsilon=epsT)
    println(termination_status(modelB))
    PhiXrob = OcalZ(blk) * kron(I(L), Hankx.H) * Grob
    PhiUrob = OcalZ(blk) * kron(I(L), Hanku.H) * Grob
    Krob = PhiUrob * inv(PhiXrob)

    Jstar = trueSystemLqrCost(blk, blqr, Kstar)
    Jnom = trueSystemLqrCost(blk, blqr, Knom)
    Jrob = trueSystemLqrCost(blk, blqr, Krob)
    
    println("(Jstar, Jnom, Jrob) = " * string((Jstar, Jnom, Jrob)))

    println("rel err of Jnom vs Jstar = " * string((Jnom - Jstar) / Jstar))

    println("rel err of Jrob (eps=0.1) vs Jstar = " * string((Jrob - Jstar) / Jstar))

    println("rel err of Jrob (eps=0.1) vs Jnom = " * string((Jrob - Jnom) / Jnom))
 
end

sanitycheck2()


(12509.683449361653, 12507.385008959936, 1433.190298532072, 1.146712465875579)
eps: 1.0250845744020884
opnorms (Hx, Hz, Hu, Hw) = (34769.45031297246, 34774.816856818186, 1335.0508324154741, 1.0250845744020884)
term stat: OPTIMAL at bisection mid = 0.5
term stat: OPTIMAL at bisection mid = 0.250005
term stat: OPTIMAL at bisection mid = 0.1250075
term stat: OPTIMAL at bisection mid = 0.06250875
term stat: OPTIMAL at bisection mid = 0.031259375
term stat: SLOW_PROGRESS at bisection mid = 0.0156346875
term stat: OPTIMAL at bisection mid = 0.02344703125
Total iterations: 7
 31.943990 seconds (24.51 M allocations: 36.569 GiB, 18.24% gc time)
OPTIMAL
(Jstar, Jnom, Jrob) = (0.37052282787719465, 0.4061346766952448, 0.4029429994452535)
rel err of Jnom vs Jstar = 0.09611242854341284
rel err of Jrob (eps=0.1) vs Jstar = 0.08749844578754039
rel err of Jrob (eps=0.1) vs Jnom = -0.007858667169132725


-----------------------------------------------

In [21]:

# A single step of the MPC
# Given x(t), figure out u(t) to derive x(t+1)
function MpcStep(blk::BlockLti, K::Matrix{Float64}, xt::Array{Float64})
    (n, m) = (blk.lti.n, lti.m)
    (A, B, sigma) = (blk.lti.A, blk.lti.B, blk.lti.sigma)
    L = blk.L
    
    @assert (m * L, n * L) == size(K)
    @assert length(xt) == n
    
    uts = K[:,1:n] * xt
    ut = uts[1:m]
    
    wt = sigma * randn(n)
    xt1 = A * xt + B * ut + wt
    
    return (xt1, ut, wt)
    
end

# Run an MPC using some K over horizon H and get the LQR costs
function MpcLoop(blk::BlockLti, K::Matrix{Float64}, x0::Array{Float64}; H=1000)
    n = blk.lti.n
    m = blk.lti.m
    L = blk.L
    (kh, kw) = size(K)
    @assert size(K) == (L * m, L * n)
    
    xhist = x0
    xt = x0
    uhist = []
    whist = []
    for t = 0:H-1
        (xt1, ut, wt) = MpcStep(blk, K, xt)
        uhist = [uhist; ut]
        whist = [whist; wt]
        xt = xt1
        if t < H - 1; xhist = [xhist; xt1] end
    end
    
    return (xhist, uhist, whist)
end

# Cost of using a particular G in the MPC loop
function MpcLoopCost(blk::BlockLti, blqr::BlockLqr, Hx::Hankel, Hu::Hankel, G, x0, H)
    PhiX = OcalZ(blk) * kron(I(L), Hx.H) * G
    PhiU = OcalZ(blk) * kron(I(L), Hu.H) * G
    K = PhiU * inv(PhiX)
    (xH, uH, _) = MpcLoop(blk, K, x0, H=H)
    bigQ = kron(I(H), blqr.lqr.Q)
    bigR = kron(I(H), blqr.lqr.R)
    sqrtBigQ = sqrt(bigQ)
    sqrtBigR = sqrt(bigR)
    J = lqr.sigma * norm(vec([sqrtBigQ * xH; sqrtBigR * uH]))
    return (J, norm(xH), norm(uH))
end
;

In [22]:
# Values taken from "L4DC Sample Complexity".ipynb
bootvals = [
#     (1, 3.9541887079997937);
#     (3, 2.311419167690131);
    (10, 1.2410479200418025);
    (31, 0.7166296851847598);
    (100, 0.39373544288332085);
    (316, 0.22195227125386172);
    (1000, 0.12497013402143313);
    (3162, 0.07103707465682507);
    (10000, 0.03887421881685629);
]

# How many (N, boot eps) values there are
Bmax = length(bootvals)

# How many independent samples per iteration
Mmax = 50

# WARNING: The expected runtime of this block
# takes about 1-2 minute per M, per B,
# So in the current configuration expect about 350 - 700 minutes!
# The Mmax value can be reduced to speed things up

# Data that is stored during runs
Jstars = zeros(Bmax, Mmax)
Jnoms = zeros(Bmax, Mmax)
JrobBs = zeros(Bmax, Mmax)
gammaBs = zeros(Bmax, Mmax)
JrobTs = zeros(Bmax, Mmax)
gammaTs = zeros(Bmax, Mmax)

normHxs = zeros(Bmax, Mmax)
normHzs = zeros(Bmax, Mmax)
normHus = zeros(Bmax, Mmax)
normHws = zeros(Bmax, Mmax)

normxstars = zeros(Bmax, Mmax)
normxnoms = zeros(Bmax, Mmax)
normxrobBs = zeros(Bmax, Mmax)
normxrobTs = zeros(Bmax, Mmax)

normustars = zeros(Bmax, Mmax)
normunoms = zeros(Bmax, Mmax)
normurobBs = zeros(Bmax, Mmax)
normurobTs = zeros(Bmax, Mmax)

# Hankel matrix dimensions
L = 10
T = 45

@assert L == 10
@assert T == 45

# Control horizon
H = 1000

# Start running a big MPC loop


println(Dates.format(now(), "HH:MM:SS") * " @ loop start")
for bi in 1:Bmax
    (N, epsilonboot) = bootvals[bi]
    
    println(Dates.format(now(), "HH:MM:SS") * " @ bi = " * string(bi))
    
    for mi = 1:Mmax
        println(Dates.format(now(), "HH:MM:SS") * " @ mi = " * string(mi))

        # Generate N random trajectories
        x0 = zeros(lti.n)
        ubar = randn(lti.m * T)
        
        avgz = zeros(lti.n * T)
        avgx = zeros(lti.n * T)
        avgu = zeros(lti.m * T)
        avgw = zeros(lti.n * T)
        
        for k = 1:N
            u = ubar
            w = lti.sigma * randn(lti.n * T)
            x = xtraj(lti, x0, u, w)
            z = ztraj(lti, x0, u)
            
            avgz = avgz + z
            avgx = avgx + x
            avgu = avgu + u
            avgw = avgw + w
        end
        
        # ... and average the N random trajectories
        avgz = avgz / N
        avgx = avgx / N
        avgu = avgu / N
        avgw = avgw / N
        
        # Toggle to use a single w trajectory
#         avgw = (lti.sigma / sqrt(N)) * randn(lti.n * T)
#         avgu = randn(lti.m * T)
#         avgx = xtraj(lti, x0, avgu, avgw)
#         avgz = ztraj(lti, x0, avgu)
        
        # Use this to build some Hankel matrices
        Hx = Hankelize(avgx, lti.n, L)
        Hz = Hankelize(avgz, lti.n, L)
        Hu = Hankelize(avgu, lti.m, L)
        Hw = Hankelize(avgw, lti.n, L)
        
        # Calculate ||Hw||
        epsilontrue = opnorm(Hw.H)
        println("(epsB, epsT) = " * string((epsilonboot, epsilontrue)))
        
        # Store some norm information on the Hankel matrices
        normHxs[bi, mi] = opnorm(Hx.H)
        normHzs[bi, mi] = opnorm(Hz.H)
        normHus[bi, mi] = opnorm(Hu.H)
        normHws[bi, mi] = opnorm(Hw.H)
        
        # Optimal controller
        (modstar, Gstar) = findGstar(blk, blqr, Hz, Hu)
        if termination_status(modstar) == MOI.OPTIMAL
            (_Jstar, _normx, _normu) = MpcLoopCost(blk, blqr, Hz, Hu, Gstar, x0, H)
            Jstars[bi, mi] = _Jstar
            normxstars[bi, mi] = _normx
            normustars[bi, mi] = _normu
            println("Jstar @ " * string((N, mi)) * " = " * string(Jstars[bi, mi]))
        else
            Jstars[bi, mi] = Inf
            normxstars[bi, mi] = Inf
            normustars[bi, mi] = Inf
            println("Jstar @ " * string((N, mi)) * " is !INFEASIBLE!") 
        end
            
        # Nominal controller
        (modnom, Gnom) = findGnom(blk, blqr, Hx, Hu)
        if termination_status(modnom) == MOI.OPTIMAL
            (_Jnom, _normx, _normu) = MpcLoopCost(blk, blqr, Hx, Hu, Gnom, x0, H)
            Jnoms[bi, mi] = _Jnom
            normxnoms[bi, mi] = _normx
            normunoms[bi, mi] = _normu
            println("Jnom @ " * string((N, mi)) * " = " * string(Jnoms[bi, mi]))
        else
            Jnoms[bi, mi] = Inf
            normxnoms[bi, mi] = Inf
            normunoms[bi, mi] = Inf
            println("Jnom @ " * string((N, mi)) * " is !INFEASIBLE!")
        end
        
        # Robust bootstrap controller
        (modrobB, GrobB, gammaB) = findGrob(blk, blqr, Hx, Hu, epsilonboot)
        if termination_status(modrobB) == MOI.OPTIMAL
            (_JrobB, _normx, _normu) = MpcLoopCost(blk, blqr, Hx, Hu, GrobB, x0, H)
            JrobBs[bi, mi] = _JrobB
            normxrobBs[bi, mi] = _normx
            normurobBs[bi, mi] = _normu
            gammaBs[bi, mi] = gammaB
            println("JrobB @ " * string((N, mi)) * " = " * string(JrobBs[bi, mi]))
        else
            JrobBs[bi, mi] = Inf
            gammaBs[bi, mi] = Inf
            normxrobBs[bi, mi] = Inf
            normurobBs[bi, mi] = Inf
            println("JrobB @ " * string((N, mi)) * " is !INFEASIBLE!")
        end
           
        # Robust true-epsilon controller
        (modrobT, GrobT, gammaT) = findGrob(blk, blqr, Hx, Hu, epsilontrue)
        if termination_status(modrobT) == MOI.OPTIMAL
            (_JrobT, _normx, _normu) = MpcLoopCost(blk, blqr, Hx, Hu, GrobT, x0, H)
            JrobTs[bi, mi] = _JrobT
            normxrobTs[bi, mi] = _normx
            normurobTs[bi, mi] = _normu
            gammaTs[bi, mi] = gammaT
            println("JrobT @ " * string((N, mi)) * " = " * string(JrobTs[bi, mi]))
        else
            JrobTs[bi, mi] = Inf
            normxrobTs[bi, mi] = Inf
            normurobTs[bi, mi] = Inf
            gammaTs[bi, mi] = Inf
            println("JrobT @ " * string((N, mi)) * " is !INFEASIBLE!")
        end
    end
end


println(Dates.format(now(), "HH:MM:SS") * " @ loop end")

13:15:00 @ loop start
13:15:00 @ bi = 1
13:15:00 @ mi = 1
(epsB, epsT) = (1.2410479200418025, 0.97390778041187)
Jstar @ (10, 1) = 1.0949793689045413
Jnom @ (10, 1) = 162105.85679090294
term stat: INFEASIBLE at bisection mid = 0.5
term stat: OPTIMAL at bisection mid = 0.749995
term stat: INFEASIBLE at bisection mid = 0.6249975
term stat: INFEASIBLE at bisection mid = 0.6874962499999999
term stat: OPTIMAL at bisection mid = 0.7187456249999999
term stat: INFEASIBLE at bisection mid = 0.7031209374999999
term stat: OPTIMAL at bisection mid = 0.71093328125
Total iterations: 7
JrobB @ (10, 1) = 2.3561513854455876
term stat: INFEASIBLE at bisection mid = 0.5
term stat: OPTIMAL at bisection mid = 0.749995
term stat: OPTIMAL at bisection mid = 0.6249975
term stat: OPTIMAL at bisection mid = 0.56249875
term stat: INFEASIBLE at bisection mid = 0.531249375
term stat: INFEASIBLE at bisection mid = 0.5468740625
term stat: SLOW_PROGRESS at bisection mid = 0.5546864062500001
Total iterations: 7
JrobT @

term stat: INFEASIBLE at bisection mid = 0.6874962499999999
term stat: OPTIMAL at bisection mid = 0.7187456249999999
term stat: OPTIMAL at bisection mid = 0.7031209374999999
term stat: SLOW_PROGRESS at bisection mid = 0.6953085937499999
Total iterations: 7
JrobB @ (10, 9) = 2.2168465708906813
term stat: INFEASIBLE at bisection mid = 0.5
term stat: OPTIMAL at bisection mid = 0.749995
term stat: OPTIMAL at bisection mid = 0.6249975
term stat: INFEASIBLE at bisection mid = 0.56249875
term stat: OPTIMAL at bisection mid = 0.5937481250000001
term stat: OPTIMAL at bisection mid = 0.5781234375000001
term stat: INFEASIBLE at bisection mid = 0.57031109375
Total iterations: 7
JrobT @ (10, 9) = 2.1309026117220022
13:27:36 @ mi = 10
(epsB, epsT) = (1.2410479200418025, 0.994496694483666)
Jstar @ (10, 10) = 1.0618994805283368
Jnom @ (10, 10) = 139595.40544376886
term stat: OPTIMAL at bisection mid = 0.5
term stat: INFEASIBLE at bisection mid = 0.250005
term stat: INFEASIBLE at bisection mid = 0.3750

term stat: OPTIMAL at bisection mid = 0.5
term stat: INFEASIBLE at bisection mid = 0.250005
term stat: INFEASIBLE at bisection mid = 0.3750025
term stat: OPTIMAL at bisection mid = 0.43750125
term stat: INFEASIBLE at bisection mid = 0.40625187500000004
term stat: INFEASIBLE at bisection mid = 0.42187656250000005
term stat: INFEASIBLE at bisection mid = 0.42968890625
Total iterations: 7
JrobT @ (10, 17) = 1.6739929954287422
13:38:16 @ mi = 18
(epsB, epsT) = (1.2410479200418025, 1.1268694838702904)
Jstar @ (10, 18) = 0.8849556802955804
Jnom @ (10, 18) = 1.5474460473369809e7
term stat: OPTIMAL at bisection mid = 0.5
term stat: INFEASIBLE at bisection mid = 0.250005
term stat: INFEASIBLE at bisection mid = 0.3750025
term stat: INFEASIBLE at bisection mid = 0.43750125
term stat: OPTIMAL at bisection mid = 0.468750625
term stat: OPTIMAL at bisection mid = 0.45312593749999996
term stat: INFEASIBLE at bisection mid = 0.44531359375
Total iterations: 7
JrobB @ (10, 18) = 1.8596258335405684
term 

term stat: SLOW_PROGRESS at bisection mid = 0.53906171875
Total iterations: 7
JrobT @ (10, 25) = 2.104472943125261
13:49:54 @ mi = 26
(epsB, epsT) = (1.2410479200418025, 1.1359314754773313)
Jstar @ (10, 26) = 1.2928429324728068
Jnom @ (10, 26) = 57.272446296143
term stat: INFEASIBLE at bisection mid = 0.5
term stat: OPTIMAL at bisection mid = 0.749995
term stat: OPTIMAL at bisection mid = 0.6249975
term stat: INFEASIBLE at bisection mid = 0.56249875
term stat: OPTIMAL at bisection mid = 0.5937481250000001
term stat: OPTIMAL at bisection mid = 0.5781234375000001
term stat: INFEASIBLE at bisection mid = 0.57031109375
Total iterations: 7
JrobB @ (10, 26) = 1.5594162923696673
term stat: INFEASIBLE at bisection mid = 0.5
term stat: OPTIMAL at bisection mid = 0.749995
term stat: OPTIMAL at bisection mid = 0.6249975
term stat: OPTIMAL at bisection mid = 0.56249875
term stat: OPTIMAL at bisection mid = 0.531249375
term stat: INFEASIBLE at bisection mid = 0.5156246875
term stat: INFEASIBLE at b

term stat: OPTIMAL at bisection mid = 0.6249975
term stat: INFEASIBLE at bisection mid = 0.56249875
term stat: OPTIMAL at bisection mid = 0.5937481250000001
term stat: OPTIMAL at bisection mid = 0.5781234375000001
term stat: OPTIMAL at bisection mid = 0.57031109375
Total iterations: 7
JrobB @ (10, 34) = 2.0891697401636415
term stat: OPTIMAL at bisection mid = 0.5
term stat: INFEASIBLE at bisection mid = 0.250005
term stat: INFEASIBLE at bisection mid = 0.3750025
term stat: OPTIMAL at bisection mid = 0.43750125
term stat: OPTIMAL at bisection mid = 0.40625187500000004
term stat: OPTIMAL at bisection mid = 0.39062718750000003
term stat: OPTIMAL at bisection mid = 0.38281484375
Total iterations: 7
JrobT @ (10, 34) = 2.019925973109172
14:03:12 @ mi = 35
(epsB, epsT) = (1.2410479200418025, 1.0767547402607844)
Jstar @ (10, 35) = 1.2570614912669604
Jnom @ (10, 35) = 2448.0758123702703
term stat: INFEASIBLE at bisection mid = 0.5
term stat: OPTIMAL at bisection mid = 0.749995
term stat: OPTIMA

...

term stat: OPTIMAL at bisection mid = 0.1250075
term stat: OPTIMAL at bisection mid = 0.06250875
term stat: OPTIMAL at bisection mid = 0.031259375
term stat: INFEASIBLE at bisection mid = 0.0156346875
term stat: OPTIMAL at bisection mid = 0.02344703125
Total iterations: 7
JrobT @ (10000, 9) = 1.6773694542467736
20:58:44 @ mi = 10
(epsB, epsT) = (0.03887421881685629, 0.03076423200433986)
Jstar @ (10000, 10) = 1.0362171136721614
Jnom @ (10000, 10) = 18.81788357489617
term stat: OPTIMAL at bisection mid = 0.5
term stat: OPTIMAL at bisection mid = 0.250005
term stat: OPTIMAL at bisection mid = 0.1250075
term stat: OPTIMAL at bisection mid = 0.06250875
term stat: OPTIMAL at bisection mid = 0.031259375
term stat: INFEASIBLE at bisection mid = 0.0156346875
term stat: OPTIMAL at bisection mid = 0.02344703125
Total iterations: 7
JrobB @ (10000, 10) = 1.994238935524523
term stat: OPTIMAL at bisection mid = 0.5
term stat: OPTIMAL at bisection mid = 0.250005
term stat: OPTIMAL at bisection mid = 0

term stat: OPTIMAL at bisection mid = 0.5
term stat: OPTIMAL at bisection mid = 0.250005
term stat: OPTIMAL at bisection mid = 0.1250075
term stat: OPTIMAL at bisection mid = 0.06250875
term stat: OPTIMAL at bisection mid = 0.031259375
term stat: INFEASIBLE at bisection mid = 0.0156346875
term stat: OPTIMAL at bisection mid = 0.02344703125
Total iterations: 7
JrobB @ (10000, 18) = 2.1942848600894065
term stat: OPTIMAL at bisection mid = 0.5
term stat: OPTIMAL at bisection mid = 0.250005
term stat: OPTIMAL at bisection mid = 0.1250075
term stat: OPTIMAL at bisection mid = 0.06250875
term stat: OPTIMAL at bisection mid = 0.031259375
term stat: INFEASIBLE at bisection mid = 0.0156346875
term stat: OPTIMAL at bisection mid = 0.02344703125
Total iterations: 7
JrobT @ (10000, 18) = 2.076890236718885
21:11:55 @ mi = 19
(epsB, epsT) = (0.03887421881685629, 0.037484452192721895)
Jstar @ (10000, 19) = 1.1916023331174486
Jnom @ (10000, 19) = 292.6717198790583
term stat: OPTIMAL at bisection mid =

term stat: OPTIMAL at bisection mid = 0.1250075
term stat: OPTIMAL at bisection mid = 0.06250875
term stat: OPTIMAL at bisection mid = 0.031259375
term stat: OPTIMAL at bisection mid = 0.0156346875
term stat: INFEASIBLE at bisection mid = 0.00782234375
Total iterations: 7
JrobT @ (10000, 26) = 1.7444336867860422
21:25:00 @ mi = 27
(epsB, epsT) = (0.03887421881685629, 0.0322190962044587)
Jstar @ (10000, 27) = 1.153860183192072
Jnom @ (10000, 27) = 1266.366506820996
term stat: OPTIMAL at bisection mid = 0.5
term stat: OPTIMAL at bisection mid = 0.250005
term stat: OPTIMAL at bisection mid = 0.1250075
term stat: OPTIMAL at bisection mid = 0.06250875
term stat: OPTIMAL at bisection mid = 0.031259375
term stat: OPTIMAL at bisection mid = 0.0156346875
term stat: INFEASIBLE at bisection mid = 0.00782234375
Total iterations: 7
JrobB @ (10000, 27) = 2.3061567075971694
term stat: OPTIMAL at bisection mid = 0.5
term stat: OPTIMAL at bisection mid = 0.250005
term stat: OPTIMAL at bisection mid = 0

term stat: OPTIMAL at bisection mid = 0.250005
term stat: OPTIMAL at bisection mid = 0.1250075
term stat: OPTIMAL at bisection mid = 0.06250875
term stat: OPTIMAL at bisection mid = 0.031259375
term stat: OPTIMAL at bisection mid = 0.0156346875
term stat: INFEASIBLE at bisection mid = 0.00782234375
Total iterations: 7
JrobB @ (10000, 35) = 2.2673167821083395
term stat: OPTIMAL at bisection mid = 0.5
term stat: OPTIMAL at bisection mid = 0.250005
term stat: OPTIMAL at bisection mid = 0.1250075
term stat: OPTIMAL at bisection mid = 0.06250875
term stat: OPTIMAL at bisection mid = 0.031259375
term stat: OPTIMAL at bisection mid = 0.0156346875
term stat: INFEASIBLE at bisection mid = 0.00782234375
Total iterations: 7
JrobT @ (10000, 35) = 1.8085084223622163
21:39:22 @ mi = 36
(epsB, epsT) = (0.03887421881685629, 0.040578097527205134)
Jstar @ (10000, 36) = 1.1501479490155413
Jnom @ (10000, 36) = 63.31398566363472
term stat: OPTIMAL at bisection mid = 0.5
term stat: OPTIMAL at bisection mid 

term stat: OPTIMAL at bisection mid = 0.06250875
term stat: OPTIMAL at bisection mid = 0.031259375
term stat: INFEASIBLE at bisection mid = 0.0156346875
term stat: OPTIMAL at bisection mid = 0.02344703125
Total iterations: 7
JrobT @ (10000, 43) = 1.5627165054281853
21:51:26 @ mi = 44
(epsB, epsT) = (0.03887421881685629, 0.03396580584462283)
Jstar @ (10000, 44) = 1.0865421808026265
Jnom @ (10000, 44) = 1562.1740210683129
term stat: OPTIMAL at bisection mid = 0.5
term stat: OPTIMAL at bisection mid = 0.250005
term stat: OPTIMAL at bisection mid = 0.1250075
term stat: OPTIMAL at bisection mid = 0.06250875
term stat: OPTIMAL at bisection mid = 0.031259375
term stat: OPTIMAL at bisection mid = 0.0156346875
term stat: INFEASIBLE at bisection mid = 0.00782234375
Total iterations: 7
JrobB @ (10000, 44) = 1.7738910786219468
term stat: OPTIMAL at bisection mid = 0.5
term stat: OPTIMAL at bisection mid = 0.250005
term stat: OPTIMAL at bisection mid = 0.1250075
term stat: OPTIMAL at bisection mid 

In [23]:
[mean(normHxs, dims=2) mean(normHzs, dims=2) mean(normHus, dims=2) mean(normHws, dims=2)]

7×4 Array{Float64,2}:
 163.513  160.586  10.8015  1.04737  
 164.63   163.204  10.6707  0.595455 
 143.371  143.662  10.5394  0.336698 
 166.065  166.107  10.4827  0.191333 
 155.101  154.948  10.7488  0.108667 
 167.6    167.769  10.6034  0.0596845
 147.745  147.748  10.6683  0.0343481

In [24]:
[median(normustars, dims=2) median(normunoms, dims=2) median(normurobBs, dims=2) median(normurobTs, dims=2)]

7×4 Array{Float64,2}:
 2.9842    25775.7  6.28677  6.11198
 3.05776   15677.3  6.15592  6.1105 
 2.96632   91640.6  5.89174  6.1593 
 2.96145   72495.9  6.15037  6.24904
 2.9848    82377.9  5.75114  5.46722
 2.95944  104703.0  5.85256  5.50498
 2.93012   28653.7  5.33406  5.15708

In [25]:
[median(Jstars, dims=2) median(Jnoms, dims=2) median(JrobBs, dims=2) median(JrobTs, dims=2)]

7×4 Array{Float64,2}:
 1.14843  25032.1        2.06874  2.07892
 1.18589  18619.1        2.02406  2.06123
 1.14214      1.02497e5  1.94577  2.0139 
 1.14566  72441.2        2.05126  2.06002
 1.15308  93200.5        1.90488  1.84666
 1.14174      1.10981e5  1.93453  1.84182
 1.1412   30700.6        1.77124  1.72772

---------------------------------------

In [26]:
# Save a bunch of results.

open("Jstars.txt", "w") do io
    writedlm(io, Jstars)
end

open("Jnoms.txt", "w") do io
    writedlm(io, Jnoms)
end

open("JrobBs.txt", "w") do io
    writedlm(io, JrobBs)
end

open("JrobTs.txt", "w") do io
    writedlm(io, JrobTs)
end

open("gammaBs.txt", "w") do io
    writedlm(io, gammaBs)
end

open("gammaTs.txt", "w") do io
    writedlm(io, gammaTs)
end


open("normxstars.txt", "w") do io
    writedlm(io, normxstars)
end


open("normxnoms.txt", "w") do io
    writedlm(io, normxnoms)
end


open("normxrobBs.txt", "w") do io
    writedlm(io, normxrobBs)
end


open("normxrobTs.txt", "w") do io
    writedlm(io, normxrobTs)
end

open("normustars.txt", "w") do io
    writedlm(io, normustars)
end


open("normunoms.txt", "w") do io
    writedlm(io, normunoms)
end


open("normurobBs.txt", "w") do io
    writedlm(io, normurobBs)
end


open("normurobTs.txt", "w") do io
    writedlm(io, normurobTs)
end
